##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Migrating feature_columns to TF2

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/migrate/migrating_feature_columns">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/migrate/migrating_feature_columns.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/migrate/migrating_feature_columns.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/migrate/migrating_feature_columns.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In [ ]:
# Temporarily install tf-nightly as the notebook depends on symbols in 2.6.
!pip uninstall -q -y tensorflow keras
!pip install -q tf-nightly

Training a model will usually come with some amount of feature preprocessing, particularly when dealing with structured data. When training an `tf.estimator.Estimator` in TF1, this feature preprocessing is done with the `tf.feature_column` API. In TF2, this preprocessing can be done directly with Keras layers, called _preprocessing layers_.

In this migration guide, you will perform some common feature transformations using both feature columns and preprocessing layers, followed by training a complete model with both APIs.

First, start with a couple of necessary imports,

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import math

and add a utility for calling a feature column for demonstration:

In [ ]:
def call_feature_columns(feature_columns, inputs):
  # This is a convenient way to call a `feature_column` outside of an estimator
  # to display its output.
  feature_layer = tf1.keras.layers.DenseFeatures(feature_columns)
  return feature_layer(inputs)

## One-hot encoding integer IDs

A common feature transformation is one-hot encoding integer inputs of a known range. Here is an example using feature columns:

In [ ]:
categorical_col = tf1.feature_column.categorical_column_with_identity(
    'type', num_buckets=3)
indicator_col = tf1.feature_column.indicator_column(categorical_col)
call_feature_columns(indicator_col, {'type': [0, 1, 2]})

Using Keras preprocessing layers, these columns can be replaced by a single `tf.keras.layers.CategoryEncoding` layer with `output_mode` set to `'one_hot'`:

In [ ]:
one_hot_layer = tf.keras.layers.CategoryEncoding(
    num_tokens=3, output_mode='one_hot')
one_hot_layer([0, 1, 2])

## One-hot encoding string data with a vocabulary

Handling string features often requires a vocabulary lookup to translate strings into indices. Here is an example using feature columns to lookup strings and then one-hot encode the indices:

In [ ]:
vocab_col = tf1.feature_column.categorical_column_with_vocabulary_list(
    'sizes',
    vocabulary_list=['small', 'medium', 'large'],
    num_oov_buckets=0)
indicator_col = tf1.feature_column.indicator_column(vocab_col)
call_feature_columns(indicator_col, {'sizes': ['small', 'medium', 'large']})

Using Keras preprocessing layers, use the `tf.keras.layers.StringLookup` layer with `output_mode` set to `'one_hot'`:

In [ ]:
string_lookup_layer = tf.keras.layers.StringLookup(
    vocabulary=['small', 'medium', 'large'],
    num_oov_indices=0,
    output_mode='one_hot')
string_lookup_layer(['small', 'medium', 'large'])

## Embedding string data with a vocabulary

For larger vocabularies, an embedding is often needed for good performance. Here is an example embedding a string feature using feature columns:

In [ ]:
vocab_col = tf1.feature_column.categorical_column_with_vocabulary_list(
    'col',
    vocabulary_list=['small', 'medium', 'large'],
    num_oov_buckets=0)
embedding_col = tf1.feature_column.embedding_column(vocab_col, 4)
call_feature_columns(embedding_col, {'col': ['small', 'medium', 'large']})

Using Keras preprocessing layers, this can be achieved by combining a `tf.keras.layers.StringLookup` layer and an `tf.keras.layers.Embedding` layer. The default output for the `StringLookup` will be integer indices which can be fed directly into an embedding.

Note that the `Embedding` layer contains trainable parameters. While the `StringLookup` layer can be applied to data inside or outside of a model, the `Embedding` must always be part of a trainable Keras model to function correctly.

In [ ]:
string_lookup_layer = tf.keras.layers.StringLookup(
    vocabulary=['small', 'medium', 'large'], num_oov_indices=0)
embedding = tf.keras.layers.Embedding(3, 4)
embedding(string_lookup_layer(['small', 'medium', 'large']))

## Complete training example

To show a complete training workflow, first prepare some data with three features of different types:

In [ ]:
features = {
    'type': [0, 1, 1],
    'size': ['small', 'small', 'medium'],
    'weight': [2.7, 1.8, 1.6],
}
labels = [1, 1, 0]
predict_features = {'type': [0], 'size': ['foo'], 'weight': [-0.7]}

Define some common constants for both TF1 and TF2 workflows:

In [ ]:
vocab = ['small', 'medium', 'large']
one_hot_dim = 3
embedding_dim = 4
weight_mean = 2.0
weight_variance = 1.0

### With feature columns

Feature columns must be passed as a list to the estimator on creation, and will be called implicitly during training.

In [ ]:
categorical_col = tf1.feature_column.categorical_column_with_identity(
    'type', num_buckets=one_hot_dim)
# Convert index to one-hot; e.g. [2] -> [0,0,1].
indicator_col = tf1.feature_column.indicator_column(categorical_col)

# Convert strings to indices; e.g. ['small'] -> [1].
vocab_col = tf1.feature_column.categorical_column_with_vocabulary_list(
    'size', vocabulary_list=vocab, num_oov_buckets=1)
# Embed the indices.
embedding_col = tf1.feature_column.embedding_column(vocab_col, embedding_dim)

normalizer_fn = lambda x: (x - weight_mean) / math.sqrt(weight_variance)
# Normalize the numeric inputs; e.g. [2.0] -> [0.0].
numeric_col = tf1.feature_column.numeric_column(
    'weight', normalizer_fn=normalizer_fn)

estimator = tf1.estimator.DNNClassifier(
    feature_columns=[indicator_col, embedding_col, numeric_col],
    hidden_units=[1])

def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

estimator.train(_input_fn)

The feature columns will also be used to transform input data when running inference on the model.

In [ ]:
def _predict_fn():
  return tf1.data.Dataset.from_tensor_slices(predict_features).batch(1)

next(estimator.predict(_predict_fn))

### With Keras preprocessing layers

Keras preprocessing layers are more flexible in where they can be called. A layer can be applied directly to tensors, used inside a `tf.data` input pipeline, or built directly into a trainable Keras model.

In this example, we will apply preprocessing layers inside a `tf.data` input pipeline. To do this, you can define a separate `tf.keras.Model` to preprocess your input features. This model is not trainable, but is a convenient way to group preprocessing layers.

In [ ]:
inputs = {
  'type': tf.keras.Input(shape=(), dtype='int64'),
  'size': tf.keras.Input(shape=(), dtype='string'),
  'weight': tf.keras.Input(shape=(), dtype='float32'),
}
outputs = {
  # Convert index to one-hot; e.g. [2] -> [0,0,1].
  'type': tf.keras.layers.CategoryEncoding(
      one_hot_dim, output_mode='one_hot')(inputs['type']),
  # Convert size strings to indices; e.g. ['small'] -> [1].
  'size': tf.keras.layers.StringLookup(vocabulary=vocab)(inputs['size']),
  # Normalize the numeric inputs; e.g. [2.0] -> [0.0].
  'weight': tf.keras.layers.Normalization(
      axis=None, mean=weight_mean, variance=weight_variance)(inputs['weight']),
}
preprocessing_model = tf.keras.Model(inputs, outputs)

You can now apply this model inside a call to `tf.data.Dataset.map`. Please note that the function passed to `map` will automatically be converted into
a `tf.function`, and usual caveats for writing `tf.function` code apply (no side effects).

In [ ]:
# Apply the preprocessing in tf.data.Dataset.map.
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
dataset = dataset.map(lambda x, y: (preprocessing_model(x), y),
                      num_parallel_calls=tf.data.AUTOTUNE)
# Display a preprocessed input sample.
next(dataset.take(1).as_numpy_iterator())

Next, you can define a separate `Model` containing the trainable layers. Note how the inputs to this model now reflect the preprocessed feature types and shapes.

In [ ]:
inputs = {
  'type': tf.keras.Input(shape=(one_hot_dim,), dtype='float32'),
  'size': tf.keras.Input(shape=(), dtype='int64'),
  'weight': tf.keras.Input(shape=(), dtype='float32'),
}
# Since the embedding is trainable, it needs to be part of the training model.
embedding = tf.keras.layers.Embedding(len(vocab), embedding_dim)
outputs = tf.keras.layers.Concatenate()([
  inputs['type'],
  embedding(inputs['size']),
  tf.expand_dims(inputs['weight'], -1),
])
outputs = tf.keras.layers.Dense(1)(outputs)
training_model = tf.keras.Model(inputs, outputs)

You can now train the `training_model` with `tf.keras.Model.fit`.

In [ ]:
# Train on the preprocessed data.
training_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))
training_model.fit(dataset)

Finally, at inference time, it can be useful to combine these separate stages into a single model that handles raw feature inputs.

In [ ]:
inputs = preprocessing_model.input
outpus = training_model(preprocessing_model(inputs))
inference_model = tf.keras.Model(inputs, outpus)

predict_dataset = tf.data.Dataset.from_tensor_slices(predict_features).batch(1)
inference_model.predict(predict_dataset)

Note: Preprocessing layers are not trainable, which allows you to apply them *asynchronously* using with `tf.data`. This has performence benefits, as you can both [prefetch](https://www.tensorflow.org/guide/data_performance#prefetching) batches with preprocessing applied, and free up any accelerators to focus on the differentiable parts of a model. However, when training performance is not important, it is sometimes simpler to add preprocessing layers directly into a complete model. This is often the case during inference, and may also be true for smaller models or when training entirely on a CPU.

## Feature column equivalence table

For reference, here is an approximate correspondence between feature columns and
preprocessing layers:<table>
  <tr>
    <th>Feature Column</th>
    <th>Keras Layer</th>
  </tr>
  <tr>
    <td>`feature_column.bucketized_column`</td>
    <td>`layers.Discretization`</td>
  </tr>
  <tr>
    <td>`feature_column.categorical_column_with_hash_bucket`</td>
    <td>`layers.Hashing`</td>
  </tr>
  <tr>
    <td>`feature_column.categorical_column_with_identity`</td>
    <td>`layers.CategoryEncoding`</td>
  </tr>
  <tr>
    <td>`feature_column.categorical_column_with_vocabulary_file`</td>
    <td>`layers.StringLookup` or `layers.IntegerLookup`</td>
  </tr>
  <tr>
    <td>`feature_column.categorical_column_with_vocabulary_list`</td>
    <td>`layers.StringLookup` or `layers.IntegerLookup`</td>
  </tr>
  <tr>
    <td>`feature_column.crossed_column`</td>
    <td>Not implemented.</td>
  </tr>
  <tr>
    <td>`feature_column.embedding_column`</td>
    <td>`layers.Embedding`</td>
  </tr>
  <tr>
    <td>`feature_column.indicator_column`</td>
    <td>`output_mode='one_hot'` or `output_mode='multi_hot'`*</td>
  </tr>
  <tr>
    <td>`feature_column.numeric_column`</td>
    <td>`layers.Normalization`</td>
  </tr>
  <tr>
    <td>`feature_column.sequence_categorical_column_with_hash_bucket`</td>
    <td>`layers.Hashing`</td>
  </tr>
  <tr>
    <td>`feature_column.sequence_categorical_column_with_identity`</td>
    <td>`layers.CategoryEncoding`</td>
  </tr>
  <tr>
    <td>`feature_column.sequence_categorical_column_with_vocabulary_file`</td>
    <td>`layers.StringLookup` or `layers.IntegerLookup`</td>
  </tr>
  <tr>
    <td>`feature_column.sequence_categorical_column_with_vocabulary_list`</td>
    <td>`layers.StringLookup` or `layers.IntegerLookup`</td>
  </tr>
  <tr>
    <td>`feature_column.sequence_numeric_column`</td>
    <td>`layers.Normalization`</td>
  </tr>
  <tr>
    <td>`feature_column.weighted_categorical_column`</td>
    <td>`layers.CategoryEncoding`</td>
  </tr>
</table>

\* `output_mode` can be passed to `layers.CategoryEncoding`, `layers.StringLookup`, and `layers.IntegerLookup`.


## Next Steps

 - For more information on keras preprocessing layers, see [the guide to preprocessing layers](https://www.tensorflow.org/guide/keras/preprocessing_layers).
 - For a more in-depth example of applying preprocessing layers to structured data, see [the structured data tutorial](https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers).